In [10]:
import os
import asyncio
import aiohttp
import pandas as pd
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv


load_dotenv()

API_KEY = os.getenv("API_KEY")
API_SECRET = os.getenv("API_SECRET")

BASE_URL = "https://data.alpaca.markets/v2/stocks"

symbol = "SPY"
number_of_days = 3


async def fetch_quotes_with_pagination(session, symbol, start, end, feed):
    headers = {
        "accept": "application/json",
        "APCA-API-KEY-ID": API_KEY,
        "APCA-API-SECRET-KEY": API_SECRET,
    }

    quotes = []
    next_page_token = None

    while True:
        params = {
            "start": start,
            "end": end,
            "limit": 10000,
            "feed": feed,
            "sort": "asc"
        }
        if next_page_token:
            params["page_token"] = next_page_token

        async with session.get(f"{BASE_URL}/{symbol}/quotes", headers=headers, params=params) as response:
            data = await response.json()
            if "quotes" not in data:
                break

            quotes.extend(data["quotes"])
            next_page_token = data.get("next_page_token")
            if not next_page_token:
                break

    return quotes


async def fetch_data(symbol, start_date, end_date):
    async with aiohttp.ClientSession() as session:
        all_quotes = []

        while start_date < end_date:
            next_date = start_date + timedelta(days=number_of_days)
            start_iso = start_date.strftime("%Y-%m-%dT%H:%M:%SZ")
            end_iso = next_date.strftime("%Y-%m-%dT%H:%M:%SZ")

            daily_quotes = await fetch_quotes_with_pagination(session, symbol, start_iso, end_iso, "sip")
            all_quotes.extend(daily_quotes)

            start_date = next_date

        df = pd.DataFrame(all_quotes)
        if not df.empty and "t" in df.columns:
            df.sort_values(by="t", inplace=True)

        return df


async def main():
    end_date = datetime.now(timezone.utc)
    start_date = end_date - timedelta(days=number_of_days)
    df = await fetch_data(symbol, start_date, end_date)

    if not df.empty:
        filename = f"{symbol}_{number_of_days}3d2_quotes.csv"
        df.to_csv(filename, index=False)


await main()